In [2]:
from tqdm import tqdm
import patterns

In [3]:
import pandas as pd
from tqdm import tqdm

# 加载 CSV 文件
df = pd.read_csv(r"C:\Users\Admin\PycharmProjects\pythonProject1\sparse_matrix_with_y.csv")

# 显示前几行数据
print(df.head())

# 假设 CSV 文件中的每一行代表一个事务
feature_columns = [f'{i}' for i in range(39)]
target_column = 'y'

# 提取特征矩阵 X 和目标向量 y
X = df[feature_columns].values
y = df[target_column].values

# 将 X 转换为事务列表格式
transactions = []
for row in tqdm(X, desc="Processing transactions"):
    transaction = [f'item_{i}' for i, val in enumerate(row) if val > 0]
    transactions.append(transaction)

# 创建包含事务和计数的 DataFrame
transactions_df = pd.DataFrame({'transaction': transactions, 'count': y})

# 显示前几行数据
print(transactions_df.head())

# 保存新的 DataFrame 到 CSV 文件
output_file_path = r"C:\Users\Admin\PycharmProjects\pythonProject1\transactions_dataset.csv"
transactions_df.to_csv(output_file_path, index=False)

print(f"Transactions dataset saved to {output_file_path}")



   0  1  2  3  4  5  6  7  8  9  ...  31  32  33  34  35  36  37  38  39   y
0  0  1  0  0  0  0  0  0  0  1  ...   0   0   0   0   0   1   0   0   0   4
1  0  0  1  1  0  0  0  0  1  1  ...   0   1   0   0   0   0   0   0   0   8
2  0  0  1  0  0  0  1  0  0  1  ...   0   1   0   1   0   0   0   0   0   5
3  0  1  1  0  1  0  0  0  0  1  ...   0   1   0   0   1   1   0   1   0  11
4  0  0  1  0  0  0  1  0  0  1  ...   0   1   0   1   1   0   0   0   1   7

[5 rows x 41 columns]


Processing transactions: 100%|██████████| 500/500 [00:00<00:00, 250018.12it/s]

                                         transaction  count
0  [item_1, item_9, item_15, item_16, item_19, it...      4
1  [item_2, item_3, item_8, item_9, item_13, item...      8
2  [item_2, item_6, item_9, item_13, item_19, ite...      5
3  [item_1, item_2, item_4, item_9, item_13, item...     11
4  [item_2, item_6, item_9, item_13, item_15, ite...      7
Transactions dataset saved to C:\Users\Admin\PycharmProjects\pythonProject1\transactions_dataset.csv


In [8]:
from skmine.itemsets import LCM
lcm = LCM(min_supp=200, n_jobs=4)
%time patterns = lcm.fit_transform(transactions)

CPU times: total: 0 ns
Wall time: 1.68 s


In [9]:
print(patterns.shape)

# 过滤出长度大于3的频繁项集
filtered_patterns = patterns[patterns.itemset.map(len) >1]

# 显示过滤后的结果
print(filtered_patterns)
print(f"Total frequent itemsets found: {len(patterns)}")


(43, 2)
                                         itemset  support
2                              [item_13, item_2]      292
4                              [item_2, item_32]      287
5                             [item_13, item_32]      289
6                     [item_13, item_2, item_32]      285
9                             [item_21, item_27]      274
12                             [item_1, item_21]      274
13                             [item_1, item_27]      274
14                    [item_1, item_21, item_27]      269
16                            [item_21, item_36]      272
17                            [item_27, item_36]      270
18                             [item_1, item_36]      270
19                    [item_1, item_21, item_36]      268
20                    [item_1, item_27, item_36]      268
21           [item_1, item_21, item_27, item_36]      267
24                            [item_19, item_24]      268
26                            [item_24, item_28]      264
27    

In [10]:
frequent_itemsets = filtered_patterns['itemset'].tolist()

# 构建新的特征矩阵
new_features = []
for transaction in transactions:
    feature_row = []
    for itemset in frequent_itemsets:
        # 判断 transaction 是否包含频繁项集
        if set(itemset).issubset(set(transaction)):
            feature_row.append(1)
        else:
            feature_row.append(0)
    new_features.append(feature_row)

# 将新的特征矩阵转换为 DataFrame
new_features_df = pd.DataFrame(new_features, columns=[f'itemset_{i}' for i in range(len(frequent_itemsets))])

# 添加目标列
new_features_df['y'] = y

# 显示前几行数据
print(new_features_df.head())
print(f"Frequent itemsets with length > 3: {len(filtered_patterns)}")


   itemset_0  itemset_1  itemset_2  itemset_3  itemset_4  itemset_5  \
0          0          0          0          0          1          1   
1          1          1          1          1          0          0   
2          1          1          1          1          0          0   
3          1          1          1          1          1          1   
4          1          1          1          1          0          0   

   itemset_6  itemset_7  itemset_8  itemset_9  ...  itemset_20  itemset_21  \
0          1          1          1          1  ...           1           1   
1          0          0          0          0  ...           1           1   
2          0          0          0          0  ...           1           1   
3          1          1          1          1  ...           1           1   
4          0          0          0          0  ...           1           1   

   itemset_22  itemset_23  itemset_24  itemset_25  itemset_26  itemset_27  \
0           1           1  

In [11]:
output_file_path = r'C:\Users\Admin\PycharmProjects\pythonProject1\Newfeatures.csv' # 请替换为你想要保存的文件路径
new_features_df.to_csv(output_file_path, index=False)